# Datapreprocessing
## 1. loading data

In [5]:
import numpy as np
import pandas as pd

filter_for_stationary=False
threshold = 1

data_name = "simdata_fast_grid"
sim_data = pd.read_csv(f"./data/raw/{data_name}.csv")
validation_data = pd.read_csv(f"./data/raw/validationdata.csv")
test_data = pd.read_csv("./data/raw/simdata_sampling_1_per_PWM.csv")

i_test = test_data[["id","iq"]].values
i_validation = validation_data[["id","iq"]].values
i_train= sim_data[["id","iq"]].values
u = np.column_stack((sim_data["ud"][1:],sim_data["uq"][1:]))
psi = sim_data[["psi_d","psi_q"]].values

## 2. normalization and filtering

In [6]:
from sklearn.preprocessing import StandardScaler

#scale currents
i_scaler = StandardScaler()
i_train_scaled = i_scaler.fit_transform(i_train)

#filer validation data
condition = i_validation[:,0]**2+i_validation[:,1]**2 < 600**2
filtered_indices = np.where(condition)[0]
i_filtered = i_validation[filtered_indices,:]
psi_filtered = validation_data[["psid","psiq"]].values[filtered_indices]
l_filterd =validation_data[["ldd","ldq","lqd","lqq"]].values[filtered_indices]

#scale validation data
i_validation_scaled = i_scaler.transform(i_filtered)
i_test_scaled = i_scaler.transform(i_test)

## 3. Assembling validation and training Data
### Training Data
X = [i_last_norm,i_norm] <br>
Y =[u,i_last,i,omega,T]
### validation Data
X =[i_norm,scale]<br>
Y=[Psi,L,R]<br>
additional_data=[i]

In [7]:
import torch
import joblib


#assemble training data
i = torch.from_numpy(i_train[:-1,:]).float()
i_next = torch.from_numpy(i_train[1:,:]).float()
u = torch.from_numpy(u).float()
omega_el = torch.tensor(sim_data["omega_el"][:-1])
delta_t = torch.tensor(sim_data["Time"][1:].values-sim_data["Time"][:-1].values)

x_train = torch.from_numpy(i_train_scaled[:-1,:]).float()
y_train = torch.column_stack([i,i_next,u,delta_t,omega_el])

if filter_for_stationary:
    i_transitions = np.column_stack((i[:-1,:],i[1:,:]))
    condition1 = np.abs(i_transitions[:,0]-i_transitions[:,2])<threshold
    condition2 = np.abs(i_transitions[:,1]-i_transitions[:,3]) < threshold
    stationary_indices = np.where(condition1&condition2)[0]
    x_train = torch.from_numpy(i_train_scaled[stationary_indices,:]).float()
    y_train = y_train[stationary_indices,:]
    print(f"{(len(stationary_indices)/len(i))*100} % are considerd stationary")
    
#assemble validation data
i_validation_scaled = torch.from_numpy(i_validation_scaled).float()
psi_validation = torch.tensor(psi_filtered)
l_validation = torch.tensor(l_filterd)
r_validation = torch.ones((len(i_validation_scaled),))*sim_data["r"][0]
i_validation = torch.from_numpy(i_filtered).float()

#assemble test data
for key in test_data.keys():
    test_data[key] = torch.from_numpy(test_data[key].values).float()
    
r_test = torch.from_numpy(test_data["r"].values).float()
omega_el_test = torch.from_numpy(test_data["r"].values).float()
psi_test = torch.from_numpy(test_data[["psi_d","psi_q"]].values).float()
l_test = torch.from_numpy(test_data[["ldd","ldq","lqd","lqq"]].values).float()
i_test_scaled = torch.from_numpy(i_test_scaled).float()

## 4. write Data to disk


In [8]:
import os
data_name = f"{data_name}_for_i_pred"
if filter_for_stationary:
    data_name = f"{data_name}_stationary"
# Define the folder path
folder_path = f"./data/clean/{data_name}"

# Check if the folder exists, otherwise create it
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created.")
else:
    print(f"Folder '{folder_path}' already exists.")

torch.save({"X":x_train,"Y":y_train},f"./data/clean/{data_name}/train.pt")
torch.save({"i_scaled":i_validation_scaled,"psi":psi_validation,"l":l_validation,"r":r_validation,"i":i_validation,"omega_el":omega_el},f"./data/clean/{data_name}/validation.pt")
torch.save({"i_scaled":i_test_scaled,"psi":psi_test,"l":l_test,"r":r_test,"i":i_test,"omega_el":omega_el_test},f"./data/clean/{data_name}/test.pt")
joblib.dump(i_scaler,f"./data/clean/{data_name}/scaler.pkl")

Folder './data/clean/simdata_fast_grid_for_i_pred' already exists.


['./data/clean/simdata_fast_grid_for_i_pred/scaler.pkl']